# step 1 import df

In [1]:
import pandas as pd
import numpy as np
import os

In [6]:
path = r'/Users/tayen/Documents/CF/Instacart Basket Analysis/02 Data/Prepared Data'

In [3]:
df_ords_prods_merge = pd.read_pickle(os.path.join(path,'df_ords_prods_labelled.pickle'))

In [26]:
df_ords_prods_merge.head()

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,_merge1,product_name,aisle_id,department_id,prices,_merge,price_range_loc,busiest_day,busiest_period_of_day,max_order,user_loyalty
0,2539329,1,1,2,8,NaN,196,1,0,both,Soda,77,7,9.0,both,mid range product,regularly days,Average orders,10,Regular customer
1,2398795,1,2,3,7,15.0,196,1,1,both,Soda,77,7,9.0,both,mid range product,lowest days,Average orders,10,Regular customer
2,473747,1,3,3,12,21.0,196,1,1,both,Soda,77,7,9.0,both,mid range product,lowest days,Most orders,10,Regular customer
3,2254736,1,4,4,7,29.0,196,1,1,both,Soda,77,7,9.0,both,mid range product,lowest days,Average orders,10,Regular customer
4,431534,1,5,4,15,28.0,196,1,1,both,Soda,77,7,9.0,both,mid range product,lowest days,Most orders,10,Regular customer


# step 2 single aggregation

In [ ]:
df_ords_prods_merge.groupby(['department_id']).agg({'order_number':['mean']})

In [7]:
df_ords_prods_merge.groupby(['department_id'])['order_number'].mean()

department_id
1     15.457838
2     17.277920
3     17.170395
4     17.811403
5     15.215751
6     16.439806
7     17.225802
8     15.340650
9     15.895474
10    20.197148
11    16.170638
12    15.887671
13    16.583536
14    16.773669
15    16.165037
16    17.665606
17    15.694469
18    19.310397
19    17.177343
20    16.473447
21    22.902379
Name: order_number, dtype: float64

In [15]:
df_ords_prods_merge.groupby(['department_id'])['order_id'].count()

department_id
1     2234743
2       36291
3     1172428
4     9479291
5      153696
6      269253
7     2688123
8       97716
9      866627
10      34573
11     447572
12     708927
13    1875369
14     703033
15    1068058
16    5398747
17     738666
18     423802
19    2887550
20    1051249
21      69145
Name: order_id, dtype: int64

# step 3

### it contains all the department rather than some of them. the means of order number across different department are closer.

# step 4 loyal label with transform() & loc()

In [11]:
# derive a new column for max_order
df_ords_prods_merge['max_order'] = df_ords_prods_merge.groupby(['user_id'])['order_number'].transform(np.max)

In [13]:
df_ords_prods_merge['max_order'].describe()

count    3.240486e+07
mean     3.305217e+01
std      2.515525e+01
min      1.000000e+00
25%      1.300000e+01
50%      2.600000e+01
75%      4.700000e+01
max      9.900000e+01
Name: max_order, dtype: float64

In [ ]:
# create lable for user loyalty

In [16]:
df_ords_prods_merge.loc[df_ords_prods_merge['max_order'] > 40, 'user_loyalty'] = 'Loyal customer'

In [21]:
df_ords_prods_merge.loc[(df_ords_prods_merge['max_order'] <= 40) & (df_ords_prods_merge['max_order'] >= 10), 'user_loyalty'] = 'Regular customer'

In [19]:
df_ords_prods_merge.loc[df_ords_prods_merge['max_order'] < 10,'user_loyalty'] = 'New customer'

In [25]:
df_ords_prods_merge['user_loyalty'].value_counts(dropna = False)

Regular customer    16649900
Loyal customer      10284093
New customer         5470866
Name: user_loyalty, dtype: int64

# step 5 

In [28]:
df_ords_prods_merge.groupby(['user_loyalty'])['prices'].mean()

user_loyalty
Loyal customer      10.386336
New customer        13.208375
Regular customer    12.561171
Name: prices, dtype: float64

### Loyal customer tend to spend less on products puchased

# step 6

In [31]:
# derive a column for avg_spending
df_ords_prods_merge['avg_spending'] = df_ords_prods_merge.groupby(['user_id'])['prices'].transform(np.mean)

In [32]:
df_ords_prods_merge['avg_spending'].describe()

count    3.240486e+07
mean     1.198023e+01
std      8.324227e+01
min      1.000000e+00
25%      7.387298e+00
50%      7.824786e+00
75%      8.254023e+00
max      2.500542e+04
Name: avg_spending, dtype: float64

In [33]:
# create label for spending range
df_ords_prods_merge.loc[df_ords_prods_merge['avg_spending'] >= 10,'spending_range'] = 'High spender'

In [34]:
df_ords_prods_merge.loc[df_ords_prods_merge['avg_spending'] < 10,'spending_range'] = 'Low spender'

In [36]:
df_ords_prods_merge['spending_range'].value_counts(dropna = False)

Low spender     31770614
High spender      634245
Name: spending_range, dtype: int64

# step 7

In [37]:
# derive a column for med_days_since
df_ords_prods_merge['med_days_since'] = df_ords_prods_merge.groupby(['user_id'])['days_since_prior_order'].transform(np.median)

In [39]:
df_ords_prods_merge['med_days_since'].describe() 

count    3.240485e+07
mean     1.039776e+01
std      7.131754e+00
min      0.000000e+00
25%      6.000000e+00
50%      8.000000e+00
75%      1.300000e+01
max      3.000000e+01
Name: med_days_since, dtype: float64

In [40]:
# create user frequency label
df_ords_prods_merge.loc[df_ords_prods_merge['med_days_since'] > 20,'user_frequency'] = 'Non-frequent customers'

In [41]:
df_ords_prods_merge.loc[(df_ords_prods_merge['med_days_since'] <= 20) & (df_ords_prods_merge['med_days_since'] >= 10),'user_frequency'] = 'Regular customer'

In [42]:
df_ords_prods_merge.loc[df_ords_prods_merge['med_days_since'] < 10,'user_frequency'] = 'Frequent user'

In [45]:
df_ords_prods_merge['user_frequency'].value_counts(dropna = False)

Frequent user             20287988
Regular customer           8480429
Non-frequent customers     3636437
NaN                              5
Name: user_frequency, dtype: int64

# step 8

In [46]:
df_ords_prods_merge.to_pickle(os.path.join(path,'df_order_prods_labels.pickle'))